In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image
import pickle
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, x, label):
        self.x = x
        self.label = label
        self.n = x.shape[0]

    def __len__(self):
        return self.n

    def __getitem__(self, idx):
        return self.x[idx], self.label[idx]

bs = 100
# Generated Dataset
train_file = './data/model/train_dataset.pkl'
test_file = './data/model/test_dataset.pkl'
with open(train_file, 'rb') as f:
    train_dataset = pickle.load(f)
with open(test_file, 'rb') as f:
    test_dataset = pickle.load(f)
    
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

In [2]:
class CVAE(nn.Module):

    def __init__(self, condi_dim, channels, latent_dim) -> None:
        super().__init__()

        # encoder
        pre_channel = 1
        modules = []
        img_length = 28
        
        for i in range(len(channels)-1):
            modules.append(
                nn.Sequential(
                    nn.Conv2d(pre_channel,
                              channels[i+1],
                              kernel_size=3,
                              stride=2,
                              padding=1),
                    nn.BatchNorm2d(channels[i+1]),
                    nn.ReLU()
                )
            )
            pre_channel = channels[i+1]
            img_length = (img_length-1)//2+1
        
        self.encoder_projection = nn.Sequential(
                nn.Linear(pre_channel * img_length * img_length + condi_dim, pre_channel * img_length * img_length),
                nn.ReLU()
        )
        
        self.encoder = nn.Sequential(*modules)
        self.mean_linear = nn.Linear(pre_channel * img_length * img_length,
                                     latent_dim)
        self.var_linear = nn.Linear(pre_channel * img_length * img_length,
                                    latent_dim)
        self.latent_dim = latent_dim
        
        # decoder
        modules = []
        self.decoder_projection = nn.Linear(
            latent_dim + condi_dim, pre_channel * img_length * img_length)
        self.decoder_input_chw = (pre_channel, img_length, img_length)

        for i in range(len(channels)-1):
            if(i == 0):
                modules.append(
                    nn.Sequential(
                        nn.ConvTranspose2d(channels[len(channels)-i-1],
                                           channels[len(channels)-i-2],
                                           kernel_size=3,
                                           stride=2,
                                           padding=1,
                                           output_padding=0),
                        nn.BatchNorm2d(channels[len(channels)-i-2]),
                        nn.ReLU()
                    )
                )
            else:
                modules.append(
                    nn.Sequential(
                        nn.ConvTranspose2d(channels[len(channels)-i-1],
                                           channels[len(channels)-i-2],
                                           kernel_size=3,
                                           stride=2,
                                           padding=1,
                                           output_padding=1),
                        nn.BatchNorm2d(channels[len(channels)-i-2]),
                        nn.ReLU()
                    )
                )
        self.decoder_layers = nn.Sequential(*modules)
        
    def decoder(self, z, c):
        z = torch.cat([z, c], dim = 1)
        z = self.decoder_projection(z)
        z = torch.reshape(z, (-1, *self.decoder_input_chw))
        decoded = self.decoder_layers(z)
        return decoded

    def forward(self, x, c):
        x = self.encoder(x)
        x = torch.flatten(x, 1)
        x = torch.cat([x, c], dim = 1)
        encoded = self.encoder_projection(x)
        mean = self.mean_linear(encoded)
        logvar = self.var_linear(encoded)
        eps = torch.randn_like(logvar)
        std = torch.exp(logvar / 2)
        z = eps * std + mean
        decoded = self.decoder(z, c)
        return decoded, mean, logvar

In [3]:
from time import time
import numpy as np

n_epochs = 200
learning_rate = 1e-2
kl_weight = 2e-5
num_class = 10

def one_hot_encode(labels, num_classes):
    one_hot_labels = np.zeros((len(labels), num_classes))
    one_hot_labels[np.arange(len(labels)), labels] = 1
    return one_hot_labels.tolist()

def elbo_loss(x, x_hat, mean, logvar):
    recons_loss = F.mse_loss(x_hat, x)
    kl_loss = -0.5 * torch.sum(1 + logvar - mean**2 - torch.exp(logvar))
    loss = (recons_loss + kl_loss * kl_weight)/len(x)
    return loss

def reconst_loss(x, x_hat):
    return F.mse_loss(x_hat, x) / len(x)

def kl_loss(mean, logvar):
    return -0.5 * torch.sum(1 + logvar - mean**2 - torch.exp(logvar)) / len(mean)

def train(device, model):
    optimizer = torch.optim.Adagrad(model.parameters(), lr=learning_rate)
    begin_time = time()
    # train
    with open('./loss.txt', 'w') as file:
        for i in range(n_epochs):
            for batch_idx, (x, label) in enumerate(train_loader):
                x = x.to(device)
                label = one_hot_encode(label, num_classes = num_class)
                label = torch.tensor(label)
                label = label.to(device)
                x_hat, mean, logvar = model(x, label)
                loss = elbo_loss(x, x_hat, mean, logvar)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
            # estimate loss
            model.eval()
            with torch.no_grad():
                each_epoch = 10
                n_samples = 100
                #train
                indices = torch.randperm(len(train_dataset))[:n_samples]
                x = torch.stack([train_dataset[i][0] for i in indices]).to(device)
                label = one_hot_encode([train_dataset[i][1] for i in indices], num_classes = num_class)
                label = torch.tensor(label).to(device)
                x_hat, mean, logvar = model(x, label)
                loss = elbo_loss(x, x_hat, mean, logvar)
                loss_recons = reconst_loss(x, x_hat)
                loss_kl = kl_loss(mean, logvar)
                if(i % each_epoch == 0):
                    print('====> Epoch: {} elbo loss: {:.7f}'.format(i, loss))
                    print('====> Epoch: {} reconst loss: {:.7f}'.format(i, loss_recons))
                    print('====> Epoch: {} kl loss: {:.7f}'.format(i, loss_kl))
                file.write(str(loss.item()) + ' ' + str(loss_recons.item()) + ' ' + str(loss_kl.item()) + ' ')
                #test
                indices = torch.randperm(len(test_dataset))[:n_samples]
                x = torch.stack([test_dataset[i][0] for i in indices]).to(device)
                label = one_hot_encode([test_dataset[i][1] for i in indices], num_classes = num_class)
                label = torch.tensor(label).to(device)
                x_hat, mean, logvar = model(x, label)
                loss = elbo_loss(x, x_hat, mean, logvar)
                loss_recons = reconst_loss(x, x_hat)
                loss_kl = kl_loss(mean, logvar)
                if(i % each_epoch == 0):
                    print('====> Epoch: {} elbo loss: {:.7f}'.format(i, loss))
                    print('====> Epoch: {} reconst loss: {:.7f}'.format(i, loss_recons))
                    print('====> Epoch: {} kl loss: {:.7f}'.format(i, loss_kl))
                file.write(str(loss.item()) + ' ' + str(loss_recons.item()) + ' ' + str(loss_kl.item()) + '\n')
            
            #time
            if(i % each_epoch == 0):
                training_time = time() - begin_time
                minute = int(training_time // 60)
                second = int(training_time % 60)
                print(f'time loss {minute}:{second}')
        
        torch.save(model, './model.pth')
        
    tot_training_time = time() - begin_time
    minute = int(tot_training_time // 60)
    second = int(tot_training_time % 60)
    print(f'total time loss {minute}:{second}')

def initialize_parameters(model):
    for param in model.parameters():
        param.data.normal_(mean=0, std=0.01)

In [4]:
def main():
    device = 'cuda:0'

    model = CVAE(condi_dim = num_class, channels = [1, 500, 500, 500], latent_dim = 5).to(device)
    initialize_parameters(model)

    train(device, model)

if __name__ == '__main__':
    main()

====> Epoch: 0 elbo loss: 0.0002382
====> Epoch: 0 reconst loss: 0.0001814
====> Epoch: 0 kl loss: 2.8390386
====> Epoch: 0 elbo loss: 0.0002455
====> Epoch: 0 reconst loss: 0.0001881
====> Epoch: 0 kl loss: 2.8707669
time loss 0:14
====> Epoch: 10 elbo loss: 0.0001866
====> Epoch: 10 reconst loss: 0.0001038
====> Epoch: 10 kl loss: 4.1406040
====> Epoch: 10 elbo loss: 0.0001818
====> Epoch: 10 reconst loss: 0.0001031
====> Epoch: 10 kl loss: 3.9378133
time loss 2:56
====> Epoch: 20 elbo loss: 0.0001753
====> Epoch: 20 reconst loss: 0.0000846
====> Epoch: 20 kl loss: 4.5334730
====> Epoch: 20 elbo loss: 0.0001849
====> Epoch: 20 reconst loss: 0.0000907
====> Epoch: 20 kl loss: 4.7086744
time loss 5:37
====> Epoch: 30 elbo loss: 0.0001787
====> Epoch: 30 reconst loss: 0.0000767
====> Epoch: 30 kl loss: 5.0955462
====> Epoch: 30 elbo loss: 0.0001759
====> Epoch: 30 reconst loss: 0.0000730
====> Epoch: 30 kl loss: 5.1475382
time loss 8:18
====> Epoch: 40 elbo loss: 0.0001748
====> Epoch: 

In [5]:
device = 'cpu'
model = torch.load('./model.pth', map_location=device)

In [6]:
latent_dim = 5

In [12]:
import torchvision
from torchvision.utils import save_image
# Generation
import random
with torch.no_grad():    
    
    noise = torch.randn(100, latent_dim).to(device)
    c = [i for i in range(10) for _ in range(10)]
    c = one_hot_encode(c, num_classes = num_class)
    c = torch.tensor(c).to(device)
    generated_imgs = model.decoder(noise, c)
    resized_image = torchvision.transforms.Resize((50, 50))(generated_imgs)
    save_image(resized_image, './pictures/genera.png', nrow=10)

In [8]:
trains_recon = []
trains_kl = []
tests_recon = []
tests_kl = []
with open('./loss.txt', 'r') as file:
    for line in file:
        parts = line.split()
        trains_recon.append(float(parts[1]))
        trains_kl.append(float(parts[2]))
        tests_recon.append(float(parts[4]))
        tests_kl.append(float(parts[5]))

mean_num = 10
trains_recon = np.mean(np.array(trains_recon[(len(trains_recon)-mean_num):])).item()
trains_kl = np.mean(np.array(trains_kl[(len(trains_kl)-mean_num):])).item()
tests_recon = np.mean(np.array(tests_recon[(len(tests_recon)-mean_num):])).item()
tests_kl = np.mean(np.array(tests_kl[(len(tests_kl)-mean_num):])).item()
print("train recon:", trains_recon, "train kl:", trains_kl, "test recon:", tests_recon, "test kl:", tests_kl)

train recon: 6.345364963635802e-05 train kl: 5.097848892211914 test recon: 6.703297985950485e-05 test kl: 5.134755039215088


In [9]:
device = 'cuda:0'
model = torch.load('./model.pth', map_location=device)

In [10]:
model.eval()
with torch.no_grad():
    num = 30000
    t = 100
    num0 = int(num/t)
    for i in range(t):
        noise = torch.randn(num0, latent_dim).to(device)
        label0 = torch.randint(low=0, high=10, size=(num0,), dtype=torch.int32)
        c = label0.tolist()
        c = one_hot_encode(c, num_classes = num_class)
        c = torch.tensor(c).to(device)
        x0 = model.decoder(noise, c)
        if(i == 0):
            x = x0
        else:
            x = torch.cat((x, x0), 0)

In [11]:
import random
from torchvision.transforms import ToPILImage
import torch_fidelity
import numpy as np
import os
import shutil

save_dir = './samples/generated'
if os.path.exists(save_dir) and os.path.isdir(save_dir):
    shutil.rmtree(save_dir)
os.makedirs(save_dir, exist_ok=True)
to_pil = ToPILImage()
for i in range(x.size(0)):
    image_tensor = x[i]
    image = to_pil(image_tensor)
    save_path = os.path.join(save_dir, f'image_{i}.png')
    image.save(save_path)

# IS and FID
metrics_dict = torch_fidelity.calculate_metrics(
    input1= './samples/generated',
    input2= './samples/base',
    cuda=True,
    isc=True,
    fid=True
)

Creating feature extractor "inception-v3-compat" with features ['2048', 'logits_unbiased']
Extracting features from input1
Looking for samples non-recursivelty in "./samples/generated" with extensions png,jpg,jpeg
Found 30000 samples
Processing samples                                                              
Extracting features from input2
Looking for samples non-recursivelty in "./samples/base" with extensions png,jpg,jpeg
Found 30000 samples
Processing samples                                                              
Inception Score: 2.4242594527240393 ± 0.017104198470361376
Frechet Inception Distance: 16.07205644801263
